In [90]:
from pywikibot.data import api
import pywikibot
import pprint

def get_items(site, itemtitle):
    params = { 'action' :'wbsearchentities' , 'format' : 'json' , 'language' : 'fr', 'type' : 'item', 'search': itemtitle}
    request = api.Request(site=site,**params)
    return request.submit()

def get_item(site, wdItem, token):
    request = api.Request(site=site,
                      action='wbgetentities',
                      format='json',
                      ids=wdItem)    
    return request.submit()

def prettyPrint(variable):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(variable)

# Login to wikidata
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
wikidataEntries = getItems(site, "history")
# Print the different Wikidata entries to the screen
prettyPrint(wikidataEntries['search'][-2]['description'])

# Print each wikidata entry as an object
#for wdEntry in wikidataEntries["search"]:
#   prettyPrint(getItem(site, wdEntry["id"], token))

'past events and their record as studied by the science of history'


  import sys



In [45]:
for c in pywikibot.ItemPage(repo, 'Q21198').get()['claims']['P31']:
    print(c.getTarget().id)

Q11862829
Q268592


In [115]:
import re

r_en = re.compile(r'(field|stud|academic|discipline|science)(y\b|ies\b|ied\b|\b|s\b)', re.IGNORECASE)

def get_academic_entity(site, search):
    items = get_items(site, search)
    for item in items['search']:
        try:
            if r_en.search(item['description']):
                return item
        except:
            pass

def get_id_from_properties(repo, entity_id, property_id='P31'):
    """yield Q id of the entity from the specific property"""
    claims = pywikibot.ItemPage(repo, entity_id).get()['claims']
    if property_id in claims:
        for item in  claims[property_id]:
            yield item.getTarget().id
    else:
        raise KeyError('Property {} is missing'.format(property_id))

print(get_academic_entity(site, 'history'))

  import sys



{'repository': '', 'id': 'Q309', 'concepturi': 'http://www.wikidata.org/entity/Q309', 'title': 'Q309', 'pageid': 496, 'url': '//www.wikidata.org/wiki/Q309', 'label': 'history', 'description': 'umbrella term covering the record of past events and their study', 'match': {'type': 'label', 'language': 'en', 'text': 'history'}}


In [119]:
len(wos)

102

In [79]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.istex
springer = db.springer

import tqdm
scopus = set()
wos = set()
sciencemetrix = set()
inist= set()
for i in tqdm.tqdm(springer.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}, 'language': 'eng'})):
            for ar, v in i['categories'].items():
                if ar == 'scopus':
                    for el in v:
                        scopus.add(el)
                if ar == 'wos':
                    for el in v:
                        wos.add(el)
                if ar == 'scienceMetrix':
                    for el in v:
                        sciencemetrix.add(el)
                if ar == 'inist':
                    for el in v:
                        inist.add(el)

51805it [00:36, 1431.44it/s]


In [117]:
for x in list(map(lambda x: x.split('- ')[1],list(inist))):
    if get_academic_entity(site, x):
        print(get_academic_entity(site, x)['label'])

  import sys



philosophy
bioethics
Pollution
electrical engineering
earth sciences
astronomy
ophthalmology
sociology
mathematics
dermatology
chemistry
nuclear physics
analytical chemistry
linguistics
applied science
microbiology
organic chemistry
Learning sciences
ethnology
physics


In [120]:
for x in list(map(lambda x: x.split('- ')[1],list(wos))):
    if get_academic_entity(site, x):
        print(get_academic_entity(site, x)['label'])

  import sys



linguistics
social science
demography
ecology
microscopy
mathematics
psychiatry
evolutionary biology
ophthalmology
cell biology
Forestry
microbiology
environmental studies
mineralogy
pediatrics
entomology
quantum thermodynamics
social work
logic
pathology
dermatology
neuroscience
economics
biology
physiology
geography
biophysics
science
botany
mechanics


In [121]:
for x in list(map(lambda x: x.split('- ')[1],list(scopus))):
    if get_academic_entity(site, x):
        print(get_academic_entity(site, x)['label'])

  import sys



logic
life sciences
environmental science
materials science
biochemistry
chemistry
cell biology
theoretical computer science
neuroscience
history and philosophy of science
mechanical engineering
pharmaceutical sciences
biophysics
microbiology
applied mathematics
Health Professions Education
dermatology
food science
Astronomy and Astrophysics Decadal Survey
physiology
psychology
chemical engineering
oceanography
ecology
physical science
environmental chemistry
demography
computer science
artificial intelligence
environmental engineering
pharmacology
Chiropractic
atmospheric sciences
Computational Mathematics and Mathematical Physics
genetics
social science
organic chemistry
ophthalmology
pedagogy
analytical chemistry
analysis
social science
engineering
mathematics
epidemiology
botany
inorganic chemistry
Pollution
Forestry
Instrumentation
philosophy
Bioengineering & Translational Medicine
Internal Medicine and Medical Investigation
soil science
histology
Medical Laboratory Technology Jou

In [123]:
for x in list(map(lambda x: x.split('- ')[1],list(sciencemetrix))):
    if get_academic_entity(site, x):
        print(get_academic_entity(site, x)['label'])

  import sys



epidemiology
Economic Theory
natural science
operations research
applied mathematics
analytical chemistry
general physics
social work
acoustics
pedagogy
evolutionary biology
clinical psychology
Business & Management Studies: An International Journal
food science
economics
ecology
physiology
chemistry
psychiatry
social science
pathology
biology
geology
Forestry
biophysics
applied science
engineering
microbiology
entomology
clinical medicine
philosophy


In [144]:
db = client.wikidata
coll = db.science

ressource = {'inist': inist, 'sciencemetrix': sciencemetrix, 'wos': wos, 'scopus': scopus}

for i, j in ressource.items():
    elements = list(map(lambda x: x.split('- ')[1],list(j)))
    for el in elements:
        if get_academic_entity(site, el):
            item = get_academic_entity(site, el)
            record = {'label': item['label'], '_id': item['id'], 'description': item['description'], 'nomenclature': [{'source': i, 'label': el}]}
            if coll.count({'_id': item['id']}) > 0:
                coll.update_one({'_id': item['id']}, {'$push':{'nomenclature': {'source': i, 'label': el}}})
            else:
                coll.insert_one(record)

  import sys



In [156]:
def get_info_from_id(site, q_id, language):
    details = pywikibot.ItemPage(site, q_id).get()
    label = details['labels'][language]
    description = details['descriptions'][language]
    record = {'_id': q_id,}
    if 'P279' in details['claims']:
        subclass_of = details['claims']

'pediatrics'

In [159]:
 pywikibot.ItemPage(site, 'Q123028').get()['descriptions']

{'en': 'field of medicine dealing with the care of children',
 'fr': 'branche de la médecine ayant trait au soin des enfants',
 'es': 'rama de la medicina que se ocupa del cuidado de los niños',
 'it': 'branca della medicina che si occupa della salute dei bambini',
 'de': 'Lehre von der Entwicklung des kindlichen und jugendlichen Organismus',
 'fi': 'lapsuusiälle ominaiset sairaudet',
 'az': 'Tibbin şöbəsi'}